In [1]:
try:
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys 
except Exception as e:
    print(e)

In [2]:
data = db.DataBase()

In [13]:
def getData(ano, formacao, limitePaginas=None):
    
    navegador = webdriver.Chrome()
    navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")

    #Tags de pesquisa
    if formacao != "doutorado":
        navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

        if formacao == "mestrado":
            navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

    navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE) #Buscar por assunto

    #Caixa de texto
    buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
    buscar.send_keys(ano)
    buscar.send_keys(Keys.ENTER)

    time.sleep(2)

    passadas = []

    janelaAtual = navegador.current_window_handle
    passadas.append(navegador.window_handles)

    #Variaveis de controle da paginação
    pagina = 1
    pag = 2

    while(True):
        #Troca de pagina
        try:
            if pagina!=1 and pag<=11 and pagina<=11:
                navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                pag = pag+1 if pag<11 else 3

            if pagina>11 and pag>2:
                navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                pag = pag+1 if pag<13 else 3
        except Exception as e:
            print(e)
            break

        #Limitador de pagínas
        if limitePaginas != None and pagina == limitePaginas:
            break

        time.sleep(2)

        for i in range(1,11):
            try:
                #Selecionar curriculo
                navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

                time.sleep(2)

                #Abrir curriculo
                navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()
            
            except:
                i -= 1
                navegador.get(driver.current_url)
                
            time.sleep(3)

            try:
                for janela in navegador.window_handles:
                    if janela not in passadas:
                        navegador.switch_to.window(janela)

            except:
                navegador.switch_to.window(janelaAtual)
                navegador.get(driver.current_url)
                i -= 1
                continue
            
            try:
                #Pegar dados 
                site = BeautifulSoup(navegador.page_source, 'html.parser')
                artigos = []

                for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                    if "2021" in artigo.text:
                        artigo = artigo.text[10:-4:]
                        text = ""
                        maior = 0
                        for s in artigo.split("."):
                            for ss in s.split(","):
                                if len(ss) > maior:
                                    text = ss
                                    maior = len(ss)
                        artigos.append(text)

                nome = site.findAll('h2', attrs={'class':['nome']})[0].text
            except:
                navegador.close()
                time.sleep(1)
                #navegador.switch_to.window(janelaAtual)
                i-=1
                continue
            
            #Inserir no banco de dados
            if artigos != []:
                data.insertData(nome, formacao, artigos, ano)

            navegador.close()

            time.sleep(1)

            navegador.switch_to.window(janelaAtual)

            #Fechar curriculo
            navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()

        data.connection.commit()
        pagina += 1
        
    navegador.quit()

In [ ]:
getData("2018", "doutorado", 100)

getData("2019", "doutorado", 100)

In [15]:
data.selectData(2018)

,primeiro_nome,nome_completo,formacao,titulo,ano
0,Paulo,Paulo Broniera Junior,doutorado,EEG-FES-Force-MMG closed-loop control systems...,2018
1,Cláudia,Cláudia Queiroz de Vasconcelos,doutorado,Avaliação da compacidade em habitações com di...,2018
2,Cláudia,Cláudia Queiroz de Vasconcelos,doutorado,PHYSICAL AND MECHANICAL CHARACTERIZATION OF P...,2018
3,Cynara,Cynara Monteiro Mariano,doutorado,Acesso aos serviços públicos e planejamento u...,2018
4,Adalberon,Adalberon Moreira de Lima Filho,doutorado,Análise dos projetos pedagógicos dos cursos d...,2018
...,...,...,...,...,...
425,Marcelo,Marcelo Siqueira Maia Vinagre Mocarzel,doutorado,Por uma nova didática: discursos e narrativas...,2018
426,Marcelo,Marcelo Siqueira Maia Vinagre Mocarzel,doutorado,Formação e inserção profissional de egressos ...,2018
427,Marcelo,Marcelo Siqueira Maia Vinagre Mocarzel,doutorado,REVISTA BRASILEIRA DE POLÍTICA E ADMINISTRAÇÃ...,2018
428,Larissa,Larissa Rocha Soares,doutorado,Software industry awareness on sustainable so...,2018


In [16]:
data.selectData(2019)

,primeiro_nome,nome_completo,formacao,titulo,ano
0,Galdino,Galdino Luiz Ramos Junior,doutorado,A inserção definitiva do direito civil e proc...,2019
1,Galdino,Galdino Luiz Ramos Junior,doutorado,QUEM SÃO OS DONOS DO PODER?,2019
2,Anderson,Anderson Mutter Teixeira,doutorado,The transmission mechanisms of macroprudentia...,2019
3,Anderson,Anderson Mutter Teixeira,doutorado,Prudential measures and their adverse effects...,2019
4,Maciel,Maciel Manoel de Queiroz,doutorado,Social capital and individual entrepreneurial...,2019
...,...,...,...,...,...
557,Edson,Edson Jansen Pedrosa de Miranda Júnior,doutorado,Plane wave expansion and extended plane wave ...,2019
558,Edson,Edson Jansen Pedrosa de Miranda Júnior,doutorado,Elastic wave propagation in metamaterial rods...,2019
559,Wellington,Wellington Douglas Guimarães Gonçalves,doutorado,Bimetallic RuPd nanoparticles in ionic liquid...,2019
560,Victor,Victor Bernardo Chabu,doutorado,BRAZILIAN JOURNAL OF PHYSICS,2019
